In [1]:
#import snscrape.modules.twitter as sntwitter, pandas as pd
import inspect, pendulum, os , requests, pandas as pd
from airflow.providers.mysql.hooks.mysql import MySqlHook
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification
#from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from airflow.models import Variable
from helper import tweet_scrapper
from bs4 import BeautifulSoup as BS
import re, calendar, asyncio, aiohttp, nest_asyncio, time, concurrent.futures, math, sys, random
from datetime import date, timedelta
from lxml import html
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
nest_asyncio.apply()

spark = SparkSession.builder.getOrCreate()
os.getcwd()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/14 13:36:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'/home/kamlesh/airflow'

# Previous stuff

In [2]:
token = None
with open('/home/kamlesh/Desktop/twitter-keys') as f:
    lines = f.readlines()
    for line in lines:
        fields = line.split(',')
        #print(fields)
        if fields[0]=='Bearer token':
            token = fields[1].strip()
print(token)
strt_dt = pendulum.now(tz='UTC').subtract(days=1).format('YYYY-MM-DDTHH:mm:ss') + '.000Z'
strt_dt

AAAAAAAAAAAAAAAAAAAAAPHZkQEAAAAAWCn5%2BCO6asCP1HC7hYg%2FplLnXHM%3D0GiBbKiZfX34h3BW6G94IxRPPk3YUcpqzTiAKRQtqGNhCxTign


'2022-12-21T14:24:46.000Z'

In [3]:
scraper = tweet_scrapper(strt_dt, token)

In [4]:
nopag_df = scraper.scrap()
nopag_df

,tweet_id,created_at,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1605930345278459904,2022-12-22 14:16:45,@yestoriches This is what I’m currently runnin...,en,0,0,TheKG89,False
1,1605929602978906112,2022-12-22 14:13:48,Today's lowest news sentiment stocks under S&a...,en,0,0,finbrain_t,False
2,1605929253157175297,2022-12-22 14:12:25,@StephenCFU_ This is what I’m currently runnin...,en,0,0,TheKG89,False
3,1605928626310221824,2022-12-22 14:09:55,I labyu so much S&amp;P500!!! Not bad for a $2...,en,0,0,CarlosMallari01,False
4,1605928321027493888,2022-12-22 14:08:43,DB comparing drawdowns in gdp vs the s&amp;p50...,en,0,0,VKMacro,False
5,1605926132062240769,2022-12-22 14:00:01,Today's highest news sentiment stocks under S&...,en,0,0,finbrain_t,False
6,1605925315380903936,2022-12-22 13:56:46,@cryptochimpanz Is simply following S&amp;P500.,en,0,0,AHigh1992,False
7,1605925180265545728,2022-12-22 13:56:14,S&amp;P500 (22-12-2022)\n\nContext is bullish...,en,1,4,chevaliernoir27,False
8,1605925040272424961,2022-12-22 13:55:40,S&amp;P500 Historical PE https://t.co/zlaYGmd8Lu,en,0,0,ksola0009,False
9,1605924691092180992,2022-12-22 13:54:17,S&amp;P500 historical earnings https://t.co/ey...,en,0,0,ksola0009,False


In [5]:
pag_df = scraper.scrap(all_results=True)
pag_df

total pages retreived: 23


,tweet_id,created_at,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1605930345278459904,2022-12-22 14:16:45,@yestoriches This is what I’m currently runnin...,en,0,0,TheKG89,False
1,1605929602978906112,2022-12-22 14:13:48,Today's lowest news sentiment stocks under S&a...,en,0,0,finbrain_t,False
2,1605929253157175297,2022-12-22 14:12:25,@StephenCFU_ This is what I’m currently runnin...,en,0,0,TheKG89,False
3,1605928626310221824,2022-12-22 14:09:55,I labyu so much S&amp;P500!!! Not bad for a $2...,en,0,0,CarlosMallari01,False
4,1605928321027493888,2022-12-22 14:08:43,DB comparing drawdowns in gdp vs the s&amp;p50...,en,0,0,VKMacro,False
...,...,...,...,...,...,...,...,...
291,1605573724194934786,2022-12-21 14:39:40,"12/21/22 #BidenBarometer: Natural Gas up 109%,...",en,0,0,KirkLindstrom,False
292,1605573022877786112,2022-12-21 14:36:53,@KangSimar @CryptoCapo_ @CryptoKaleo What is y...,en,0,0,Ghostbanned7,False
293,1605571977284124672,2022-12-21 14:32:44,@elonmusk @28delayslater The spread in the las...,en,0,2,dario_munzo,False
294,1605571784916762624,2022-12-21 14:31:58,"@NorthmanTrader Imho, S&amp;P500 roughly 4000 ...",en,0,0,hikinashii,False


In [19]:
pag_df[(pag_df['like_cnt']>=20)|(pag_df['verified'])]

,tweet_id,created_at,tweet,lang,retweet_cnt,like_cnt,username,verified
137,1605736544941195265,2022-12-22 01:26:40,What decoupled US Market last night from us??\...,en,3,44,Nupurkunia,True
159,1605715318516731908,2022-12-22 00:02:19,Goldman Sachs forecast S&amp;P500 to 4000 in 2...,en,4,8,ForexLive,True
197,1605676610929446912,2022-12-21 21:28:30,A Wall Street rebound overnight saw S&amp;P500...,en,0,0,KevanSangster,True
225,1605641424891846656,2022-12-21 19:08:41,$SPY $SPX Just remember when you short the S&a...,en,12,102,Mr_Derivatives,False
233,1605631300110426112,2022-12-21 18:28:27,$SPY $TSLA \n\nElon says buy Tesla here and sh...,en,5,120,Mr_Derivatives,False
261,1605601776987623433,2022-12-21 16:31:08,Riddles warned us that Japano will trigger dom...,en,1,21,SharedWisdom,False
265,1605594568698654720,2022-12-21 16:02:30,3 stocks that could do well into 2023 after hi...,en,4,36,StephenCFU_,False
289,1605574790391242756,2022-12-21 14:43:54,@28delayslater He's saying the inflation adjus...,en,0,20,CostcoAvocado,False
295,1605570058310209536,2022-12-21 14:25:06,"History of financial media offering 100 daily,...",en,13,56,SethCL,False


In [9]:
pag_df[:10]

,tweet_id,created_at,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1605930345278459904,2022-12-22 14:16:45,@yestoriches This is what I’m currently runnin...,en,0,0,TheKG89,False
1,1605929602978906112,2022-12-22 14:13:48,Today's lowest news sentiment stocks under S&a...,en,0,0,finbrain_t,False
2,1605929253157175297,2022-12-22 14:12:25,@StephenCFU_ This is what I’m currently runnin...,en,0,0,TheKG89,False
3,1605928626310221824,2022-12-22 14:09:55,I labyu so much S&amp;P500!!! Not bad for a $2...,en,0,0,CarlosMallari01,False
4,1605928321027493888,2022-12-22 14:08:43,DB comparing drawdowns in gdp vs the s&amp;p50...,en,0,0,VKMacro,False
5,1605926132062240769,2022-12-22 14:00:01,Today's highest news sentiment stocks under S&...,en,0,0,finbrain_t,False
6,1605925315380903936,2022-12-22 13:56:46,@cryptochimpanz Is simply following S&amp;P500.,en,0,0,AHigh1992,False
7,1605925180265545728,2022-12-22 13:56:14,S&amp;P500 (22-12-2022)\n\nContext is bullish...,en,1,4,chevaliernoir27,False
8,1605925180265545728,2022-12-22 13:56:14,S&amp;P500 (22-12-2022)\n\nContext is bullish...,en,1,4,chevaliernoir27,False
9,1605925040272424961,2022-12-22 13:55:40,S&amp;P500 Historical PE https://t.co/zlaYGmd8Lu,en,0,0,ksola0009,False


In [58]:
def create_url(pag_token=None):
   base_url = "https://api.twitter.com/2/tweets/search/recent"
   start_date = pendulum.now(tz='UTC').subtract(days=1).format('YYYY-MM-DDTHH:mm:ss') + '.000Z'
   query =  "lang%3Aen%20%22S%26P500%22%20-is%3Aretweet" 
   max_results = 10
                  
   url_templated = '''{}?query={}&start_time={}&max_results={}'''
   fields = '''&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified'''
   if pag_token!=None:
      url = url_templated+'&pagination_token={}'+fields
      url = url.format(base_url, query, start_date, max_results, pag_token)
   else:
      url = url_templated+fields
      url = url.format(base_url, query, start_date, max_results)

   #url = url_templated.format(base_url, query_params['query'], query_params['start_time'], query_params['max_results'])

   #&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified'''.format('q', 't1', '10')
   return url



'https://api.twitter.com/2/tweets/search/recent?query=lang%3Aen%20%22S%26P500%22%20-is%3Aretweet&start_time=2022-12-21T12:35:22.000Z&max_results=10'

In [ ]:
query = "lang%3Aen%20%22S%26P500%22%20-is%3Aretweet" #lang:en "S&P500" -is:retweet 
#start_time=2022-12-14T00:00:00.000Z
url = '''https://api.twitter.com/2/tweets/search/recent?query={}&start_time={}&max_results={}&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified'''

#start_date = pendulum.now().subtract(days=1).to_rfc3339_string()
start_date = pendulum.now(tz='UTC').subtract(days=1).format('YYYY-MM-DDTHH:mm:ss') + '.000Z' 
resp = requests.get(url=url.format(query, start_date, max_tweets), headers={'Authorization': 'Bearer {}'.format(token)})
if resp.status_code==200:
    pass
else:
    print(resp.json())
    raise Exception('Something went wrong!!')

In [44]:
resp.url

'https://api.twitter.com/2/tweets/search/recent?query=lang%3Aen+%22S%26P500%22+-is%3Aretweet&start_time=2022-12-21T12%3A11%3A40.000Z&max_results=10&tweet.fields=context_annotations%2Ccreated_at%2Clang%2Cpublic_metrics&expansions=author_id&user.fields=verified'

In [45]:
pendulum.now(tz='UTC').subtract(days=1).format('YYYY-MM-DDTHH:mm:ss') + '.000Z'

'2022-12-21T12:12:42.000Z'

In [20]:
len(resp.json()['data'])

10

In [22]:
resp.json()['meta']['next_token']

'b26v89c19zqg8o3fqk11onw4j6ja6d8oh5aga824qzcl9'

In [24]:
url.format(query, start_date, max_tweets)

'https://api.twitter.com/2/tweets/search/recent?query=lang%3Aen%20%22S%26P500%22%20-is%3Aretweet&start_time=2022-12-21T11:17:33.000Z&max_results=10&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified'

In [ ]:
resp1 = requests.get(url=url.format(query, start_date, max_tweets), headers={'Authorization': 'Bearer {}'.format(token)})

In [8]:
type(pendulum.datetime(2022,12,19))

pendulum.datetime.DateTime

In [9]:
pendulum.datetime(2022,12,19, tz='Europe/Amsterdam')

DateTime(2022, 12, 19, 0, 0, 0, tzinfo=Timezone('Europe/Amsterdam'))

In [7]:
type(pendulum.datetime(2022,12,19, tz='Europe/Amsterdam'))

pendulum.datetime.DateTime

AAAAAAAAAAAAAAAAAAAAAPHZkQEAAAAAWCn5%2BCO6asCP1HC7hYg%2FplLnXHM%3D0GiBbKiZfX34h3BW6G94IxRPPk3YUcpqzTiAKRQtqGNhCxTign


In [7]:
pendulum.now(tz='UTC').subtract(days=1).format('YYYY-MM-DDTHH:mm:ss') + '.000Z' #%Y-%m-%dT%H:%M:%S.%fZ

'2022-12-17T13:15:46.000Z'

In [8]:
def scrap_tweets(token, max_tweets=10):
    query = "lang%3Aen%20%22S%26P500%22%20-is%3Aretweet" #lang:en "S&P500" -is:retweet 
    #start_time=2022-12-14T00:00:00.000Z
    url = '''https://api.twitter.com/2/tweets/search/recent?query={}&start_time={}&max_results={}&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified'''

    #start_date = pendulum.now().subtract(days=1).to_rfc3339_string()
    start_date = pendulum.now(tz='UTC').subtract(days=1).format('YYYY-MM-DDTHH:mm:ss') + '.000Z' 
    resp = requests.get(url=url.format(query, start_date, max_tweets), headers={'Authorization': 'Bearer {}'.format(token)})
    if resp.status_code==200:
        pass
    else:
        print(resp.json())
        raise Exception('Something went wrong!!')

    cols_tweets = ['tweet_id', 'auth_id', 'time', 'tweet', 'lang', 'retweet_cnt', 'like_cnt']
    cols_users = ['auth_id', 'username', 'verified']
    tweets = [[tweet_data['id'], tweet_data['author_id'], pendulum.parse(tweet_data['created_at']).to_datetime_string(), tweet_data['text'], tweet_data['lang'], 
              tweet_data['public_metrics']['retweet_count'], tweet_data['public_metrics']['like_count']] for tweet_data in resp.json()['data']]
    users = [[user_data['id'], user_data['username'], user_data['verified']] for user_data in resp.json()['includes']['users']]

    tweet_df = pd.DataFrame(tweets, columns=cols_tweets)
    users_df = pd.DataFrame(users, columns=cols_users)

    return tweet_df.merge(users_df, how='left', on='auth_id').drop(columns=['auth_id'])    

In [9]:
final_df = scrap_tweets(bearer_tok)
final_df

,tweet_id,time,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1604462703099117568,2022-12-18 13:04:52,@mcjamez Many BIG S&amp;P500 stocks down YTD &...,en,0,0,RyanCaveman,False
1,1604462338643726336,2022-12-18 13:03:25,@AlwaysStocks @sizetrades look at breakdown an...,en,0,0,Ron01911,False
2,1604462258872258560,2022-12-18 13:03:06,"🇺🇸Btc's Correlation\n\nBTC：$16,793 ↑\nS&amp;P5...",en,0,0,btc_status,False
3,1604443138109620225,2022-12-18 11:47:07,Economists are saying it's just as bad as 2000...,en,0,2,Honeystocks1,False
4,1604442173461635073,2022-12-18 11:43:17,"Last week, the 10 microcap stock picks perform...",en,0,0,SmartTraderAI,False
5,1604438238705635328,2022-12-18 11:27:39,@marcus_keong @AsrulNasirudin Wahed’s HLAL (th...,en,0,1,douvay_,False
6,1604437700047605761,2022-12-18 11:25:31,Weekly S&amp;P500 ChartStorm - 18 December 202...,en,0,0,mrussell222,False
7,1604431685722181632,2022-12-18 11:01:37,@dokkodian @InnocentSin13 Fed members stopped ...,en,0,1,MrnllMtt,False
8,1604427819681038338,2022-12-18 10:46:15,We will wait for a rise in the 1st base of the...,en,0,1,daytradingmc,False
9,1604423054888558592,2022-12-18 10:27:19,@AltCryptoGems You know nothing but fraud. BTC...,en,0,0,Bob88488488,False


In [122]:
# url = '''https://api.twitter.com/2/tweets/search/recent?\
# query=lang%3Aen%20%22S%26P500%22&start_time=2022-12-14T00:00:00.000Z&max_results=10&\
# tweet.fields=created_at,lang,public_metrics,text&user.fields=username,verified"
# '''
url = '''https://api.twitter.com/2/tweets/search/recent?\
query=lang%3Aen%20%22S%26P500%22&start_time=2022-12-14T00:00:00.000Z&max_results=10&\
tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified
'''

print(url)

https://api.twitter.com/2/tweets/search/recent?query=lang%3Aen%20%22S%26P500%22&start_time=2022-12-14T00:00:00.000Z&max_results=10&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified



In [150]:
#url = '''https://api.twitter.com/2/tweets/search/recent?query=lang%3Aen%20%22S%26P500%22&start_time=2022-12-14T00:00:00.000Z&max_results=10&tweet.fields=created_at,lang,public_metrics,text&user.fields=username,verified'''
url = '''https://api.twitter.com/2/tweets/search/recent?query=lang%3Aen%20%22S%26P500%22%20-is%3Aretweet&start_time=2022-12-14T00:00:00.000Z&max_results=10&tweet.fields=context_annotations,created_at,lang,public_metrics&expansions=author_id&user.fields=verified'''

#query = "%22S%26P500%22"
#access_token = "AAAAAAAAAAAAAAAAAAAAAPHZkQEAAAAASBl9YDC5yy8Zv2fn7KFfbHxkxHA%3Dj9tkVBGHAriNYpvrOsBwAKX76jY6NIjYIgwYkiwC4QZhd21iXH"
#url = 'https://api.twitter.com/2/tweets/search/recent?query={}'.format(query)

r = requests.get(url=url, headers={'Authorization': 'Bearer {}'.format(bearer_tok)})
r

<Response [200]>

In [151]:
r.json()

{'data': [{'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '910658928291274754',
   'text': 'adding long S&amp;P500 200X laverage $390',
   'id': '1603422018342252545',
   'edit_history_tweet_ids': ['1603422018342252545'],
   'lang': 'en',
   'created_at': '2022-12-15T16:09:33.000Z'},
  {'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '446481468',
   'text': 'Uhh Ohh, yall seeing the technical patterns being drawn out on S&amp;P500 and Nasdaq?\n\nhttps://t.co/f9zGxzwJts https://t.co/50rHDTUCsj',
   'id': '1603421188906156032',
   'edit_history_tweet_ids': ['1603421188906156032'],
   'lang': 'en',
   'created_at': '2022-12-15T16:06:16.000Z'},
  {'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '1100436921191358464',
   'text': '@jaykaeppel 3/ and in relation to where the c

In [130]:
r.json().keys()

dict_keys(['data', 'includes', 'meta'])

In [143]:
r.json()['includes']['users'] #1537099392485937152 1475915681887571974 1475915681887571974

[{'verified': False,
  'id': '1537099392485937152',
  'name': 'Michael Francis',
  'username': 'PrydaCapital'},
 {'verified': False,
  'id': '1258146842925244416',
  'name': 'shmu28',
  'username': 'shmu28'},
 {'verified': False,
  'id': '1582157778684108800',
  'name': 'Thomas Luca',
  'username': 'ThomasL48552611'},
 {'verified': False,
  'id': '872839233232408576',
  'name': 'Rene Sellmann',
  'username': 'ReneSellmann'},
 {'verified': False,
  'id': '1571150141544964097',
  'name': 'Basket News',
  'username': 'basketnews_io'},
 {'verified': False,
  'id': '1070762267526991872',
  'name': 'Tomasz Schopenhauer',
  'username': 'AHigh1992'},
 {'verified': False,
  'id': '3161080382',
  'name': 'John',
  'username': 'andyxu0623'},
 {'verified': False,
  'id': '1508845257546371080',
  'name': 'wehaslove',
  'username': 'wehasloves2'},
 {'verified': False,
  'id': '173863158',
  'name': 'Blue.TY',
  'username': 'SereneINU'},
 {'verified': False,
  'id': '1475915681887571974',
  'name': '

In [145]:
r.json()['data'][-1].keys()

dict_keys(['lang', 'public_metrics', 'id', 'text', 'author_id', 'context_annotations', 'edit_history_tweet_ids', 'created_at'])

In [181]:
cols_tweets = ['id', 'auth_id', 'time', 'tweet', 'lang', 'retweet_cnt', 'like_cnt']
cols_users = ['auth_id', 'username', 'verified']
tweets = [[tweet_data['id'], tweet_data['author_id'], tweet_data['created_at'], tweet_data['text'], tweet_data['lang'], tweet_data['public_metrics']['retweet_count'], 
          tweet_data['public_metrics']['like_count']] for tweet_data in r.json()['data']]
users = [[user_data['id'], user_data['username'], user_data['verified']] for user_data in r.json()['includes']['users']]

# for tweet_data in r.json()['data']:
#     #print(i)
#     #print(type(i['created_at']))
#     #break
#     tweets.append([tweet_data['id'], tweet_data['created_at'], user_data['username'], user_data['verified'], tweet_data['text'], tweet_data['lang'], 
#                  tweet_data['public_metrics']['retweet_count'], tweet_data['public_metrics']['like_count']])
tweet_df = pd.DataFrame(tweets, columns=cols_tweets)
users_df = pd.DataFrame(users, columns=cols_users)
display(tweet_df)
users_df

,id,auth_id,time,tweet,lang,retweet_cnt,like_cnt
0,1603422018342252545,910658928291274754,2022-12-15T16:09:33.000Z,adding long S&amp;P500 200X laverage $390,en,0,0
1,1603421188906156032,446481468,2022-12-15T16:06:16.000Z,"Uhh Ohh, yall seeing the technical patterns be...",en,0,0
2,1603419723550654466,1100436921191358464,2022-12-15T16:00:26.000Z,@jaykaeppel 3/ and in relation to where the co...,en,0,0
3,1603419108300800004,62666453,2022-12-15T15:58:00.000Z,Legend Jeremy Grantham sees S&amp;P500 crashin...,en,0,2
4,1603418946908069891,2843981466,2022-12-15T15:57:21.000Z,Current portfolio by size:\n$CLPT\n$XPEL\n$CSU...,en,0,5
5,1603418784131424256,1582157778684108800,2022-12-15T15:56:42.000Z,Today looked like it could have been 4th time ...,en,0,1
6,1603418706406776833,1100436921191358464,2022-12-15T15:56:24.000Z,"@jaykaeppel 2/ In order to find some ""order"" i...",en,0,1
7,1603417842661167104,1352528242473902081,2022-12-15T15:52:58.000Z,S&amp;P500 breadth. 24/500 rising https://t.co...,en,0,0
8,1603416986876186625,833154196727689218,2022-12-15T15:49:34.000Z,Just a quick reminder:\nVolume will be crazy t...,en,22,189
9,1603416075349147648,602811570,2022-12-15T15:45:57.000Z,the S&amp;P500 topped out around 3900 in early...,en,0,11


,auth_id,username,verified
0,910658928291274754,SERGIO81110,False
1,446481468,Carrier_Trading,False
2,1100436921191358464,MarkMap4,False
3,62666453,danspena,False
4,2843981466,maxwellhouse99,False
5,1582157778684108800,ThomasL48552611,False
6,1352528242473902081,hughesanalytics,False
7,833154196727689218,TeslaBoomerMama,False
8,602811570,RevShark,True


In [183]:
tweet_df.merge(users_df, how='left', on='auth_id').drop(columns=['auth_id'])

,id,time,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1603422018342252545,2022-12-15T16:09:33.000Z,adding long S&amp;P500 200X laverage $390,en,0,0,SERGIO81110,False
1,1603421188906156032,2022-12-15T16:06:16.000Z,"Uhh Ohh, yall seeing the technical patterns be...",en,0,0,Carrier_Trading,False
2,1603419723550654466,2022-12-15T16:00:26.000Z,@jaykaeppel 3/ and in relation to where the co...,en,0,0,MarkMap4,False
3,1603419108300800004,2022-12-15T15:58:00.000Z,Legend Jeremy Grantham sees S&amp;P500 crashin...,en,0,2,danspena,False
4,1603418946908069891,2022-12-15T15:57:21.000Z,Current portfolio by size:\n$CLPT\n$XPEL\n$CSU...,en,0,5,maxwellhouse99,False
5,1603418784131424256,2022-12-15T15:56:42.000Z,Today looked like it could have been 4th time ...,en,0,1,ThomasL48552611,False
6,1603418706406776833,2022-12-15T15:56:24.000Z,"@jaykaeppel 2/ In order to find some ""order"" i...",en,0,1,MarkMap4,False
7,1603417842661167104,2022-12-15T15:52:58.000Z,S&amp;P500 breadth. 24/500 rising https://t.co...,en,0,0,hughesanalytics,False
8,1603416986876186625,2022-12-15T15:49:34.000Z,Just a quick reminder:\nVolume will be crazy t...,en,22,189,TeslaBoomerMama,False
9,1603416075349147648,2022-12-15T15:45:57.000Z,the S&amp;P500 topped out around 3900 in early...,en,0,11,RevShark,True


In [177]:
r1 = requests.get("https://api.twitter.com/2/tweets/1603418706406776833?tweet.fields=author_id&expansions=author_id", 
                headers={'Authorization': 'Bearer {}'.format(bearer_tok)})
r1.json()

{'data': {'edit_history_tweet_ids': ['1603418706406776833'],
  'text': '@jaykaeppel 2/ In order to find some "order" in all of it, start with the monitoring of the "monthly" S&amp;P500 price / moving average configuration on a "fixed" reference date,',
  'id': '1603418706406776833',
  'author_id': '1100436921191358464'},
 'includes': {'users': [{'id': '1100436921191358464',
    'name': 'Mark Map',
    'username': 'MarkMap4'}]}}

In [149]:
pendulum.parse("2022-12-15T15:35:34.000Z")

DateTime(2022, 12, 15, 15, 35, 34, tzinfo=Timezone('UTC'))

In [10]:
t = [[1,4,5], [1, 7, 0], [1, 0, 0]]
sorted(t, key = lambda x: x[1], reverse=True)

[[1, 7, 0], [1, 4, 5], [1, 0, 0]]

In [56]:
inspect.signature(sntwitter.TwitterSearchScraper)

<Signature (query, cursor=None, top=False, **kwargs)>

In [65]:
def tweet_scraper(limit=10, break_limit=1000):
    query = """"S&P500" lang:en until:2022-12-14 since:2022-12-13"""
    tweets = []
    mostliked = []
    itr = 0
    cols = ['Dt', 'ID', 'User', 'Verified', 'RetweetCnt','LikeCnt', 'Tweet']
    for tweet in tqdm(sntwitter.TwitterSearchScraper(query).get_items(), total=break_limit):
        if itr == break_limit:
            print("should break now!!")
            break
        else:
            if (tweet.user.verified and tweet.likeCount>=10):
                #print('date: ', tweet.date)
                tweets.append([tweet.date, tweet.id, tweet.user.username, tweet.user.verified, tweet.retweetCount, tweet.likeCount, tweet.content])
            elif tweet.likeCount>=100:
                mostliked.append([tweet.date, tweet.id, tweet.user.username, tweet.user.verified, tweet.retweetCount, tweet.likeCount, tweet.content])
            else:
                pass
        itr += 1
    
    mostliked = sorted(mostliked, key=lambda x: x[5], reverse=True) 

    #print('before length check')
    #display(pd.DataFrame(tweets, columns=cols))
    print('***INFO***length of tweets: {} and most-liked tweets:{}'.format(len(tweets), len(mostliked)))

    if len(tweets)<limit:
        for i in mostliked[int(len(tweets)-limit):]:
            tweets.append(i)

    #print('final dataframe')
    return pd.DataFrame(tweets, columns=cols)

[2022-12-15 01:39:58,164] {twitter.py:368} INFO - Retrieving scroll page None
[2022-12-15 01:39:58,166] {base.py:168} INFO - Retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%22S%26P500%22+lang%3Aen+until%3A2022-12-14+since%3A2022-12-13&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel
[2022-12-15 01:40:00,689] {base.py:184} INFO - Retrieved https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&inclu

,Dt,ID,User,Verified,RetweetCnt,LikeCnt,Tweet
0,2022-12-13 21:57:39+00:00,1602784843657527297,cyrilXBT,False,95,404,The future S&amp;P500 of crypto: Stacks Financ...
1,2022-12-13 21:00:34+00:00,1602770479441661953,tier10k,False,11,186,[DB] US Close\n\nS&amp;P500: +0.71%\nNasdaq: +...
2,2022-12-13 17:44:11+00:00,1602721057248624648,Silver_Gold_U,False,2,5,S&amp;P500 tapped 4100 on the nose...now tappi...
3,2022-12-13 21:28:06+00:00,1602777407463002115,erikhagar,False,0,20,Tom Lee at FundStrat sees huge upside for the ...
4,2022-12-13 16:21:36+00:00,1602700274963042304,spotgamma,False,17,126,The risk of buying options with ultra high imp...
5,2022-12-13 12:51:29+00:00,1602647396743348224,chevaliernoir27,False,0,16,S&amp;P500 (13-12-2022)\n\nContext is bullish...
6,2022-12-13 14:57:08+00:00,1602679014778441728,Afrectz,False,0,4,"With a forecast of 7.3%, in November inflation..."
7,2022-12-13 09:08:35+00:00,1602591302427807744,AnkurShah47,False,0,2,S&amp;P500 performance till Nov 30th:\n1-month...
8,2022-12-13 11:03:47+00:00,1602620293981290498,FactorDAO,False,4,29,Did someone say @arbitrum ecosystem indices?\n...
9,2022-12-13 11:22:14+00:00,1602624935641452545,TraderHwk,False,0,2,#us500 S&amp;P500\nShort from PRZ point &amp; ...


In [2]:
mysql_hook = MySqlHook(mysql_conn_id = 'mysql_conn', schema='Airflow')
engine = mysql_hook.get_sqlalchemy_engine()

[2022-12-23 15:37:19,617] {base.py:73} INFO - Using connection ID 'mysql_conn' for task execution.


In [13]:
sql_query="""CREATE TABLE IF NOT EXISTS raw_tweets (
            id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            tweet_id BIGINT,
            time DATETIME,
            tweet TEXT,
            lang VARCHAR(15),
            retweet_cnt INT,
            like_cnt INT,
            username VARCHAR(50),
            verified BOOLEAN
            );"""
mysql_hook.run(sql=sql_query)

[2022-12-18 14:25:36,186] {base.py:73} INFO - Using connection ID 'mysql_conn' for task execution.
[2022-12-18 14:25:36,187] {sql.py:364} INFO - Running statement: CREATE TABLE IF NOT EXISTS raw_tweets (
            id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            tweet_id BIGINT,
            time DATETIME,
            tweet TEXT,
            lang VARCHAR(15),
            retweet_cnt INT,
            like_cnt INT,
            username VARCHAR(50),
            verified BOOLEAN
            );, parameters: None
[2022-12-18 14:25:36,247] {sql.py:373} INFO - Rows affected: 0


In [15]:
final_df.to_sql(name='raw_tweets', con=engine, if_exists='append', index=False)

10

In [63]:
#mysql_hook.run('SHOW COLUMNS FROM raw_tweets;', handler=pd.DataFrame)
df = pd.read_sql("SELECT * FROM polar_tweets;", con=engine)
df
#mysql_hook.get_pandas_df("SELECT * FROM raw_tweets;")

,id,tweet_id,created_at,tweet,polarity,retweet_cnt,like_cnt,username,verified
0,1,1606326741231771663,2022-12-23 16:31:53,@BluSuitDillon The two classic alarms for a re...,negative,0,0,OneOver2Pi,0
1,2,1606326376545615873,2022-12-23 16:30:26,"Trading update: Friday December 23, 2022\nS&am...",neutral,0,0,AlBrooksTrader,0
2,3,1606325704311742465,2022-12-23 16:27:46,"@gc22gc An overlay of the S&amp;P500, Dow and ...",negative,0,0,kirktoecho,0
3,4,1606323175872159746,2022-12-23 16:17:43,ProShares UltraShort S&amp;P500 $NYSEARCA:SDS ...,neutral,0,0,WKRBNews,0
4,5,1606322867439747072,2022-12-23 16:16:30,"Twitter-investor-verse, you all know that the ...",positive,0,20,GilmanHill,1
...,...,...,...,...,...,...,...,...,...
269,270,1605969291802996744,2022-12-22 16:51:31,$SSO - ProShares Ultra S&amp;P500 declares qua...,neutral,0,0,MarketCurrents,0
270,271,1605968780462473222,2022-12-22 16:49:29,Portofolio Management Question 1: \nAs a recen...,positive,0,0,FreelanceWriti6,0
271,272,1605967983494381580,2022-12-22 16:46:19,@Tweak_TV @Julia5Lynch5 @elonmusk @CramerTrack...,neutral,0,0,Natsoph2,0
272,273,1605966080073089024,2022-12-22 16:38:45,@saxena_puru Have you considered the companies...,neutral,0,0,wei_w_toronto,0


In [64]:
fil_df = df[(df['verified']==1)|(df['like_cnt']>=10)].reset_index(drop=True)
fil_df

,id,tweet_id,created_at,tweet,polarity,retweet_cnt,like_cnt,username,verified
0,5,1606322867439747072,2022-12-23 16:16:30,"Twitter-investor-verse, you all know that the ...",positive,0,20,GilmanHill,1
1,15,1606311975419527168,2022-12-23 15:33:13,I dont give direct market advice but ill say t...,negative,5,51,SullyCNBC,1
2,48,1606261985577996289,2022-12-23 12:14:34,"""We expect to see earnings revisions come down...",neutral,0,2,SamRo,1
3,60,1606215390291320833,2022-12-23 09:09:25,1⃣AGORERO+ NYSE\n2⃣COPPOCK S&amp;P500\n3⃣BUNKE...,neutral,0,12,JavierAlfayate,0
4,74,1606156758195396608,2022-12-23 05:16:26,European/US closing levels - 22/12/22 - FTSE 7...,neutral,1,12,truemagic68,0
5,83,1606133914594004994,2022-12-23 03:45:40,❤💛💚💙\n\nMy Best-Case scenario for #Crypto is t...,positive,0,23,yourfriendSOMMI,0
6,90,1606108146262573056,2022-12-23 02:03:16,A rare December \nNegative for S&amp;P500 \n(M...,negative,5,81,iamrakeshbansal,0
7,191,1606062406211235841,2022-12-22 23:01:31,Friends don't let friends look at the S&amp;P500.,neutral,0,12,HistoryBoomer,0
8,206,1606048420812029953,2022-12-22 22:05:57,"#ES_F, 4h\n\nTwo tactical levels for S&amp;P50...",neutral,16,170,yuriymatso,0
9,207,1606048268605014016,2022-12-22 22:05:20,“Two things - a double whammy. Product purchas...,neutral,4,47,MilMileBattery,0


In [67]:
fil_df['tweet'].iloc[0]

"Twitter-investor-verse, you all know that the S&amp;P500 is up 7% QTD, right?\n\nI know it's been a tough year, but keep perspective and your chins up\n\nS&amp;P 500 annualized return for the past 3 years is 7.6% and past 5 is 9.3% - inline w long term historical averages\n\nIt's not so bad!"

In [57]:
df1 = pd.DataFrame({'col1':[1, 2, 3 ], 'col2':['a','a','b'], 'auth_id':['a1', 'a2','a2']})
df2 = pd.DataFrame({'auth_id':['a1', 'a1','a2'], 'veri':['t','t', 'f']}).drop_duplicates(subset=['auth_id'])
display(df1)
df2

,col1,col2,auth_id
0,1,a,a1
1,2,a,a2
2,3,b,a2


,auth_id,veri
0,a1,t
2,a2,f


# ETNews analysis
<!-- <span style="color:red">Start from here for ET news analysis</span> -->

In [2]:
''' 
data = spark.sparkContext.parallelize([1,2,3,4,5,6,7,8], 2)
print('num partitions: ', data.getNumPartitions())
print('first element: ', data.first())
print('before: ',data.collect())
data.map(lambda numbers: type(numbers)).collect() #1 + numbers
spark.sparkContext.parallelize([1,2,3,4], 4).glom().collect()
data.mapPartitionsWithIndex(lambda idx, x: ('idx-{}'.format(idx), [i for i in x])).glom().collect()
'''
df = spark.read.option("header",True).csv('ET_News')
print('total partitions: ', df.rdd.getNumPartitions())
#test_df = df.limit(5)
#df.show(3)
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 3g"

total partitions:  67


In [6]:
df.write.option("compression", "gzip").csv("ET_NewsCompressed")

## summarize news

In [3]:
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [5]:
config.id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}

In [8]:
#test_df.withColumn('polarity', test_df.News)

In [10]:
class nlp_task():
    def __init__(self):
        self.model_name = "ProsusAI/finbert"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def ranking(self, text):
        encoded_input = self.tokenizer(text, return_tensors='pt')
        output = self.model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores) 
        ranking = ranking[::-1]
        return ranking
nlp = nlp_task()

In [29]:
for text in df.limit(3).collect():
    print('*'*2, text.News)

** New Delhi: Copper prices edged higher by 0.11 per cent to Rs 449.40 per kg in futures trade on Friday as participants raised bets, tracking positive global cues. On the Multi Commodity Exchange, copper contracts for July delivery rose by 50 paise, or 0.11 per cent, to Rs 449.40 per kg in a business turnover of 743 lots. A firm trend overseas and pick-up in demand at the spot market mainly led to the rise in copper prices, analysts said. Zinc Zinc futures traded marginally lower at Rs 192.25 per kg on Friday as speculators reduced exposure amid subdued demand at the spot market. Zinc contracts for August delivery fell by 80 paise, or 0.41 per cent, to Rs 192.25 per kg at the Multi Commodity Exchange, clocking a business turnover of 2,758 lots. Similarly, the metal contracts for September delivery edged lower by Re 1, or 0.52 per cent, to Rs 193 per kg in a business turnover of 8 lots. Traders said the weakness in zinc prices at futures trade was mostly due to cutting down of bets by 

In [26]:
for text in df.limit(3).collect():
    encoded_input = tokenizer(text.News, return_tensors='pt')
    
    output = model(**encoded_input)
    print('[INFO]reached')
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    score_dict = {}
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        score_dict[l] = np.round(float(s), 4)
        #print(f"{i+1}) {l} {np.round(float(s), 4)}")
    #print('printing labels')
    print('***', text.News)
    print('='*3)
    print(scores)
    print(score_dict)
    #nlp.ranking(df.first().News)

[INFO]reached
*** New Delhi: Copper prices edged higher by 0.11 per cent to Rs 449.40 per kg in futures trade on Friday as participants raised bets, tracking positive global cues. On the Multi Commodity Exchange, copper contracts for July delivery rose by 50 paise, or 0.11 per cent, to Rs 449.40 per kg in a business turnover of 743 lots. A firm trend overseas and pick-up in demand at the spot market mainly led to the rise in copper prices, analysts said. Zinc Zinc futures traded marginally lower at Rs 192.25 per kg on Friday as speculators reduced exposure amid subdued demand at the spot market. Zinc contracts for August delivery fell by 80 paise, or 0.41 per cent, to Rs 192.25 per kg at the Multi Commodity Exchange, clocking a business turnover of 2,758 lots. Similarly, the metal contracts for September delivery edged lower by Re 1, or 0.52 per cent, to Rs 193 per kg in a business turnover of 8 lots. Traders said the weakness in zinc prices at futures trade was mostly due to cutting d

RuntimeError: The size of tensor a (553) must match the size of tensor b (512) at non-singleton dimension 1

In [27]:
config.id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}

In [36]:
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
score_dict = {}
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    score_dict[l] = np.round(float(s), 4)
    #print(f"{i+1}) {l} {np.round(float(s), 4)}")
#print('printing labels')
score_dict

{'negative': 0.9456, 'positive': 0.0284, 'neutral': 0.026}

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

In [51]:
def sentiment(tweet):
    new_text = []
    for t in tweet.split(" "):
        t = '@user' if t.startswith('@') and len(t)>1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    text = " ".join(new_text)
    #print('tweet: ', tweet)
    #print('cleaned tweet: ', text)
    # text = "Covid cases are increasing fast!"
    # text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    score_dict = {}
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        score_dict[l] = np.round(float(s), 4)
        #print(f"{i+1}) {l} {np.round(float(s), 4)}")
    #print('printing labels')
    return score_dict


In [34]:
df

,id,tweet_id,time,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1,1604462703099117568,2022-12-18 13:04:52,@mcjamez Many BIG S&amp;P500 stocks down YTD &...,en,0,0,RyanCaveman,0
1,2,1604462338643726336,2022-12-18 13:03:25,@AlwaysStocks @sizetrades look at breakdown an...,en,0,0,Ron01911,0
2,3,1604462258872258560,2022-12-18 13:03:06,"🇺🇸Btc's Correlation\n\nBTC：$16,793 ↑\nS&amp;P5...",en,0,0,btc_status,0
3,4,1604443138109620225,2022-12-18 11:47:07,Economists are saying it's just as bad as 2000...,en,0,2,Honeystocks1,0
4,5,1604442173461635073,2022-12-18 11:43:17,"Last week, the 10 microcap stock picks perform...",en,0,0,SmartTraderAI,0
5,6,1604438238705635328,2022-12-18 11:27:39,@marcus_keong @AsrulNasirudin Wahed’s HLAL (th...,en,0,1,douvay_,0
6,7,1604437700047605761,2022-12-18 11:25:31,Weekly S&amp;P500 ChartStorm - 18 December 202...,en,0,0,mrussell222,0
7,8,1604431685722181632,2022-12-18 11:01:37,@dokkodian @InnocentSin13 Fed members stopped ...,en,0,1,MrnllMtt,0
8,9,1604427819681038338,2022-12-18 10:46:15,We will wait for a rise in the 1st base of the...,en,0,1,daytradingmc,0
9,10,1604423054888558592,2022-12-18 10:27:19,@AltCryptoGems You know nothing but fraud. BTC...,en,0,0,Bob88488488,0


In [48]:
df.drop(columns=['test_col'], inplace=True)
df.head(1)
#%timeit df['test_col'] = df.apply(lambda x: 2*x['id'], axis=1)
#%timeit df['test_col'] = [2*i for i in df['id']]

,id,tweet_id,time,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1,1604462703099117568,2022-12-18 13:04:52,@mcjamez Many BIG S&amp;P500 stocks down YTD &...,en,0,0,RyanCaveman,0


In [52]:
df.drop(columns=['polarity'], inplace=True)
df.head(1)

,id,tweet_id,time,tweet,lang,retweet_cnt,like_cnt,username,verified
0,1,1604462703099117568,2022-12-18 13:04:52,@mcjamez Many BIG S&amp;P500 stocks down YTD &...,en,0,0,RyanCaveman,0


In [53]:
df['polarity'] = [sentiment(tweet) for tweet in df['tweet']]
df

,id,tweet_id,time,tweet,lang,retweet_cnt,like_cnt,username,verified,polarity
0,1,1604462703099117568,2022-12-18 13:04:52,@mcjamez Many BIG S&amp;P500 stocks down YTD &...,en,0,0,RyanCaveman,0,"{'negative': 0.8246, 'neutral': 0.1618, 'posit..."
1,2,1604462338643726336,2022-12-18 13:03:25,@AlwaysStocks @sizetrades look at breakdown an...,en,0,0,Ron01911,0,"{'neutral': 0.6236, 'negative': 0.3104, 'posit..."
2,3,1604462258872258560,2022-12-18 13:03:06,"🇺🇸Btc's Correlation\n\nBTC：$16,793 ↑\nS&amp;P5...",en,0,0,btc_status,0,"{'neutral': 0.8066, 'negative': 0.1052, 'posit..."
3,4,1604443138109620225,2022-12-18 11:47:07,Economists are saying it's just as bad as 2000...,en,0,2,Honeystocks1,0,"{'negative': 0.8601, 'neutral': 0.1276, 'posit..."
4,5,1604442173461635073,2022-12-18 11:43:17,"Last week, the 10 microcap stock picks perform...",en,0,0,SmartTraderAI,0,"{'negative': 0.5691, 'neutral': 0.4021, 'posit..."
5,6,1604438238705635328,2022-12-18 11:27:39,@marcus_keong @AsrulNasirudin Wahed’s HLAL (th...,en,0,1,douvay_,0,"{'neutral': 0.8878, 'positive': 0.0665, 'negat..."
6,7,1604437700047605761,2022-12-18 11:25:31,Weekly S&amp;P500 ChartStorm - 18 December 202...,en,0,0,mrussell222,0,"{'neutral': 0.9324, 'positive': 0.0575, 'negat..."
7,8,1604431685722181632,2022-12-18 11:01:37,@dokkodian @InnocentSin13 Fed members stopped ...,en,0,1,MrnllMtt,0,"{'neutral': 0.7261, 'negative': 0.2496, 'posit..."
8,9,1604427819681038338,2022-12-18 10:46:15,We will wait for a rise in the 1st base of the...,en,0,1,daytradingmc,0,"{'neutral': 0.8564, 'positive': 0.1336, 'negat..."
9,10,1604423054888558592,2022-12-18 10:27:19,@AltCryptoGems You know nothing but fraud. BTC...,en,0,0,Bob88488488,0,"{'negative': 0.8834, 'neutral': 0.1056, 'posit..."


In [70]:
df[['polarity', 'tweet_id']]

,polarity,tweet_id
0,"{'negative': 0.8246, 'neutral': 0.1618, 'posit...",1604462703099117568
1,"{'neutral': 0.6236, 'negative': 0.3104, 'posit...",1604462338643726336
2,"{'neutral': 0.8066, 'negative': 0.1052, 'posit...",1604462258872258560
3,"{'negative': 0.8601, 'neutral': 0.1276, 'posit...",1604443138109620225
4,"{'negative': 0.5691, 'neutral': 0.4021, 'posit...",1604442173461635073
5,"{'neutral': 0.8878, 'positive': 0.0665, 'negat...",1604438238705635328
6,"{'neutral': 0.9324, 'positive': 0.0575, 'negat...",1604437700047605761
7,"{'neutral': 0.7261, 'negative': 0.2496, 'posit...",1604431685722181632
8,"{'neutral': 0.8564, 'positive': 0.1336, 'negat...",1604427819681038338
9,"{'negative': 0.8834, 'neutral': 0.1056, 'posit...",1604423054888558592


In [56]:
sentiment(df['tweet'][9])

{'negative': 0.8834, 'neutral': 0.1056, 'positive': 0.011}

In [74]:
Variable.get("twitter_bearer_token")

'AAAAAAAAAAAAAAAAAAAAAPHZkQEAAAAAWCn5%2BCO6asCP1HC7hYg%2FplLnXHM%3D0GiBbKiZfX34h3BW6G94IxRPPk3YUcpqzTiAKRQtqGNhCxTign'

In [68]:
utc_time = pendulum.now(tz='UTC').to_date_string()
utc_time

'2022-12-18'

In [69]:
"SELECT * FROM raw_tweets WHERE `time` > DATE_SUB('{}', INTERVAL 1 DAY);".format(utc_time)

"SELECT * FROM raw_tweets WHERE `time` > DATE_SUB('2022-12-18', INTERVAL 1 DAY);"

In [37]:
local_tz = pendulum.timezone("Europe/Amsterdam")
local_tz

Timezone('Europe/Amsterdam')

In [44]:
pendulum.now().subtract(days=1).to_date_string()

'2022-12-13'

In [35]:
days_ago(1)

/tmp/ipykernel_24863/1349323584.py:1 RemovedInAirflow3Warning: Function `days_ago` is deprecated and will be removed in Airflow 3.0. You can achieve equivalent behavior with `pendulum.today('UTC').add(days=-N, ...)`

datetime.datetime(2022, 12, 13, 0, 0, tzinfo=Timezone('UTC'))